In [15]:
from langgraph.graph import StateGraph , START , END
from langchain_ollama import ChatOllama
from langchain_core.messages import BaseMessage , HumanMessage
from typing import Annotated , TypedDict
from langgraph.checkpoint.memory import MemorySaver

In [16]:
from langgraph.graph.message import add_messages

class ChatState(TypedDict):
    
    messages : Annotated[list[BaseMessage],add_messages]
    
    

In [17]:
llm = ChatOllama(model="gemma:2b")

def chat_node(state:ChatState):

    messages = state['messages']
    response = llm(messages)
    
    return {'messages': [response]}

In [24]:
graph = StateGraph(ChatState)
checkpointer = MemorySaver()

graph.add_node('chat_node',chat_node)

graph.add_edge(START , 'chat_node')
graph.add_edge('chat_node' , END)

chatbot = graph.compile(checkpointer=checkpointer)

In [20]:
initial_state = {
    'messages': [HumanMessage("What is the capital of Pakistan?")]
}

# chatbot.invoke(initial_state)['messages'][-1].content

In [26]:
thread_id = '1'
while True:
    
    user_message = input("Write Something...")
    
    print("User: ",user_message)
    
    if user_message.strip().lower() in ['exit','quit','bye']:
        break
    
    config = {'configurable': {'thread_id': thread_id}}
    
    response = chatbot.invoke({
        'messages': [HumanMessage(content = user_message)]
    } ,config=config)
    
    print('AI: ', response['messages'][-1].content)

User:  bye
